In [123]:
## Importo librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


In [124]:
## leo los dos archivos
trainlimpio= pd.read_csv('train_con_eda')
test= pd.read_parquet ('test.parquet')

#### Realizo con test, las misma operaciones que realice en train

In [125]:
## Borro las columnas lat y long
borrar= ['lat', 'long']
test.drop(columns= borrar, inplace= True)
## Completo los NaN para el modelo
test['laundry_options'] = test.laundry_options.fillna(test.laundry_options.mode()[0])
test['parking_options'] = test.parking_options.fillna(test.parking_options.mode()[0])
## Borro las columnas que no son para el analisis
Otro_borrar= ['id', 'url', 'region_url', 'image_url', 'description', 'state']
test.drop(columns= Otro_borrar, inplace= True)
## Cambio el tipo de dato
test['baths']=test['baths'].astype (int)

#### Ahora trabajo con las categoricas de test

In [126]:
categoricas= ['type', 'laundry_options', 'parking_options', 'region']
# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder

#Veo las categoricas
print(test[categoricas].head())

# CreoLabelEncoder : le
le = LabelEncoder()

# Aplico LabelEncoder a las columnas
test[categoricas] = test[categoricas].apply(lambda x: le.fit_transform(x))

# Veo las 5 filas de las categoricas
print(test[categoricas].head())

           type  laundry_options     parking_options                   region
0  manufactured      w/d in unit  off-street parking                ann arbor
1     apartment      w/d hookups  off-street parking                  vermont
2     apartment      w/d in unit             carport                ann arbor
3     apartment      w/d in unit     attached garage  fort collins / north CO
4     apartment  laundry on site  off-street parking          charlottesville
   type  laundry_options  parking_options  region
0     8                4                4       9
1     0                3                4     370
2     0                4                1       9
3     0                4                0     112
4     0                1                4      53


#### Reviso nuevamente el tipo de datos

In [127]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38498 entries, 0 to 38497
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   region                   38498 non-null  int32
 1   type                     38498 non-null  int32
 2   sqfeet                   38498 non-null  int64
 3   beds                     38498 non-null  int64
 4   baths                    38498 non-null  int32
 5   cats_allowed             38498 non-null  int64
 6   dogs_allowed             38498 non-null  int64
 7   smoking_allowed          38498 non-null  int64
 8   wheelchair_access        38498 non-null  int64
 9   electric_vehicle_charge  38498 non-null  int64
 10  comes_furnished          38498 non-null  int64
 11  laundry_options          38498 non-null  int32
 12  parking_options          38498 non-null  int32
dtypes: int32(5), int64(8)
memory usage: 3.1 MB


#### Reviso el shape de los dos datsets; tienen la diferencia de la columna category_price

In [128]:
trainlimpio.shape

(148880, 14)

In [129]:
test.shape

(38498, 13)

#### ahora escalamos los datos numericos de ambos

In [130]:
from sklearn.preprocessing import StandardScaler
scl = StandardScaler()
scl.fit(trainlimpio[['sqfeet']]) 

trainlimpio['sqfeet'] = scl.transform(trainlimpio[['sqfeet']]) 


In [131]:
from sklearn.preprocessing import StandardScaler
scl = StandardScaler()
scl.fit(test[['sqfeet']]) 

test['sqfeet'] = scl.transform(test[['sqfeet']]) 


#### Comienzo con el modelo; voy a utilizar todos los features, ya que considero lo mas adecuado para el analisis

In [132]:
## Importo Librerias
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score



In [133]:
## Declaro las variables del train
X= trainlimpio[['type', 'sqfeet', 'beds', 'baths', 'cats_allowed', 'dogs_allowed', 'smoking_allowed','wheelchair_access', 'electric_vehicle_charge', 'comes_furnished', 'laundry_options', 'parking_options', 'region']]
y= trainlimpio[['category_price']]


In [134]:
## Hago un train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(119104, 13) (119104, 1) (29776, 13) (29776, 1)


In [135]:
## Instancio y entreno el modelo
from sklearn.metrics import f1_score
clf = DecisionTreeClassifier(max_depth =100, random_state = 42)
clf.fit(X_train,y_train)


DecisionTreeClassifier(max_depth=100, random_state=42)

#### Ahora voy a trabajar con test

In [137]:
## Declaro mi x_test que tiene las mismas columnas que el train
X_test= test[['type', 'sqfeet', 'beds', 'baths', 'cats_allowed', 'dogs_allowed', 'smoking_allowed','wheelchair_access', 'electric_vehicle_charge', 'comes_furnished', 'laundry_options', 'parking_options', 'region']]


In [138]:
## Obtengo mi y_predict con la variabale declarada anteriormente
y_pred = clf.predict(X_test)

In [139]:
## Lo paso a un dataframe
y_predi= pd.DataFrame(y_pred, columns=['pred'])

In [116]:
## lo exporto
y_predi.to_csv('SoleGon.csv', index=False)

In [140]:
## Cuento las lineas para chequear
y_predi.value_counts()

pred
0       20769
1       17729
dtype: int64